Nora Cécile Rosel Zaballos - 27/05/2025\
First attempt, code from MediaPipe and GoogleDevelopers sites.

In [ ]:
#It worked in jupyterlab, but I cannot access the webcam on this remote server.
import cv2
import mediapipe as mp

#Initialize MediaPipe drawing and hands modules
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands

#For static images:
IMAGE_FILES = [] #Image file paths

#Create a Hands object with specific settings
with mp_hands.Hands(
    static_image_mode=True, #If set to false, input as a video stream
    max_num_hands=2,
    min_detection_confidence=0.5) as hands: #Min. confidence value for the detection to be considered successful
    
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.flip(cv2.imread(file), 1) #Read and flip image horizontally around y axis
    #Convert the BGR image to RGB before processing
    results = hands.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB)) #Run detection

    #Print handedness and draw hand landmarks on the image
    print('Handedness:', results.multi_handedness) #Each hand is composed of label (string of "Left" or "Right") and score (probability of the predicted handedness (score, 1-score))
    #!!! Note that handedness is determined assuming the input image is mirrored, i.e., taken with a front-facing/selfie camera with images flipped horizontally. If it is not the case, please swap the handedness output in the application around y axis
    
    if not results.multi_hand_landmarks: #x and y normalized to 1, z is the landmark depth with the depth at the wrist being the origin, the smaller the value the closer the landmark is to the camera
      continue #Skip if no hands detected
        
    image_height, image_width, _ = image.shape
    annotated_image = image.copy()
      
    for hand_landmarks in results.multi_hand_landmarks:
      print('hand_landmarks:', hand_landmarks)
      print(
          f'Index finger tip coordinates: (',
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x * image_width}, '
          f'{hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y * image_height})'
      )

      #Draw landmarks
      mp_drawing.draw_landmarks(
          annotated_image,
          hand_landmarks,
          mp_hands.HAND_CONNECTIONS,
          mp_drawing_styles.get_default_hand_landmarks_style(),
          mp_drawing_styles.get_default_hand_connections_style())

    #Save annotated image
    cv2.imwrite(
        '/tmp/annotated_image' + str(idx) + '.png', cv2.flip(annotated_image, 1))
      
    #Draw hand world landmarks
    if not results.multi_hand_world_landmarks: #World coordinates of the 21 landmarks, 3D and in meters, with the origin at the hand's approximate geometric center
      continue
        
    for hand_world_landmarks in results.multi_hand_world_landmarks:
      mp_drawing.plot_landmarks(
        hand_world_landmarks, mp_hands.HAND_CONNECTIONS, azimuth=5)

#For webcam input:
cap = cv2.VideoCapture(0) #Open webcam

with mp_hands.Hands(
    model_complexity=1, #Landmark accuracy as well as inference latency go up with model complexity, default 1, higher accuracy
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands: #Min. confidence from the landmark-tracking, otherwise hand detections will be invoked automatically on the next input image
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      #If loading a video, use 'break' instead of 'continue'.
      continue

    #To improve performance, optionally mark the image as not writeable to pass by reference
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image) #Run detection

    #Draw the hand annotations on the image
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
      
    if results.multi_hand_landmarks:
      for hand_landmarks in results.multi_hand_landmarks:
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
          
    #Flip the image horizontally for a selfie-view display
    cv2.imshow('MediaPipe Hands', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
        
cap.release()